# sub sample by density

In [1]:
import pickle
import load
import numpy as np
import matplotlib.pyplot as plt
from tree import halomodule as hmo
import numpy.lib.recfunctions as rcfuncs

In [2]:
wdir = "/home/hoseung/Work/data/Horizon-AGN/"
tdir = "GalaxyMaker/"
print(wdir)

/home/hoseung/Work/data/Horizon-AGN/


In [109]:
gcat = hmo.Halo(nout=782, is_gal=True, base=wdir)

gcat.data = gcat.data[gcat.data["m"] > 1e10]
ngal = len(gcat.data)

x = gcat.data["x"]
y = gcat.data["y"]
z = gcat.data["z"]

ngal = len(x)

x_rank = np.empty(ngal, int)
y_rank = np.empty(ngal, int)
z_rank = np.empty(ngal, int)

straight_ind = np.arange(ngal)

x_ind = np.argsort(x)
y_ind = np.argsort(y)
z_ind = np.argsort(z)

x_rank[x_ind] = straight_ind
y_rank[y_ind] = straight_ind 
z_rank[z_ind] = straight_ind

[Halo.load_info] loading info
[Halo.load_info] nout = 782, base =/home/hoseung/Work/data/Horizon-AGN/


In [227]:
d5=[]
N5=[]

Ncut = 10000
Rcut = 0.1
method = 2 # method 2 is slightly faster.


for i, gal in enumerate(gcat.data):
    xg,yg,zg = x_rank[i], y_rank[i],z_rank[i]
    if method == 1:
        x_ok = np.abs(x_rank - xg) < Ncut
        y_ok = np.abs(y_ind - yg) < Ncut
        z_ok = np.abs(z_ind - zg) < Ncut
        all_ok = np.where(x_ok * y_ok * z_ok)[0]    
    elif method == 2:
        multiply = 1
        while True:
            x_ok = np.abs(x - gal["x"]) < (0.05 * multiply)
            y_ok = np.abs(y[x_ok] - gal["y"]) < (0.05 * multiply)                
            all_ok = np.abs(z[x_ok][y_ok] - gal["z"]) < (0.05 * multiply)
            if sum(all_ok) > 5:
                break
            else:
                multiply *=2
                print("again")
        
        all_ok = straight_ind[x_ok][y_ok][all_ok]
    
    dist = np.sqrt(np.square(gal["x"] - x[all_ok]) + 
                   np.square(gal["y"] - y[all_ok]) + 
                   np.square(gal["z"] - z[all_ok]))
    
    d5.append(sorted(dist)[5] * gcat.info.pboxsize) # 0 = itself.
    N5.append(sum(dist < 5/gcat.info.pboxsize))
    if i % 2000 == 0:
        print("{:.1f} % done".format( 100 * i / ngal))
    #print(len(all_ok), d5, N5)

0.0 % done
7.2 % done
14.3 % done
21.5 % done
28.7 % done
35.8 % done
43.0 % done
again
50.2 % done
57.3 % done
64.5 % done
again
again
71.7 % done
again
again
78.8 % done
86.0 % done
93.2 % done


In [228]:
plt.scatter(x[all_ok], y[all_ok])
plt.show()

In [161]:
fatherID = pickle.load(open(wdir + tdir + "fatherID.pickle", "rb"))
fatherIDx = pickle.load(open(wdir + tdir + "fatherIDx.pickle", "rb"))
fatherMass = pickle.load(open(wdir + tdir + "fatherMass.pickle", "rb"))
tt = pickle.load(open(wdir + tdir + "Tree.pickle", "rb"))

t_now = tt[tt['zred'] == tt["zred"][-1]] # zred = 0.017xx?

In [229]:
d5 = np.array(d5)
N5 = np.array(N5)
#rcfuncs.append_fields(gcat.data, ["D5", "N5"], data=[d5, N5], usemask=False)
pickle.dump((gcat.data["id"], d5, N5), open(wdir + "catalog_density_d5_m10.pickle", "wb"))

In [176]:
ids, d5, N5 = pickle.load(open(wdir + "catalog_density_d5_m10.pickle", "rb"))

In [230]:
ind_chunks = np.array_split(np.argsort(N5), 10)
for i in range(10):
    ids = gcat.data["id"][ind_chunks[i]]
    final_gals = [t_now['idx'][gid-1] for gid in ids] # Every final galaxy has (short or long) tree.

    np.savetxt(wdir + "density_" + str(i * 5) + "percentile_IDs_d5_m10.txt", (ids, final_gals), fmt='%10d')

In [231]:
def extract_main_tree(t, idx, fatherID, fatherMass):
    """ 
    Extracts 
    """
    import numpy as np
    t_now = t[idx]
    nstep = t_now["nstep"]
    nouts = [nstep]
    atree = np.zeros(nstep + 1, dtype=t.dtype)
    atree[0]=t_now

    for i in range(1, nstep + 1): 
        try:
            id_father = fatherID[idx]
            if len(id_father) > 1:
                mass_father = fatherMass[idx]
                id_father_main = id_father[np.argmax(mass_father)]
                t_father = t[id_father_main-1]
                idx = t_father["idx"]
                atree[i]=t_father
                nouts.append(nstep)
            else:
                break
        except:
            break
    return atree


In [233]:
zred_complete = 0.5 # tree starting from earlier than
for i in range(10):
    alltrees=[]
    ids, final_gals = np.genfromtxt(wdir + "density_" + str(i * 5) + "percentile_IDs_d5_m10.txt", dtype=int)
    for final_idx in final_gals:
        maintree = extract_main_tree(tt, final_idx, fatherIDx, fatherMass)
        # Add main progenitor tag
        if max(maintree["zred"]) > zred_complete:
            alltrees.append(maintree)
    pickle.dump(np.array(alltrees), open(wdir +"tree_by_density_" +str(i*5) + "_percentile_d5_m10.pickle", "wb"))
